In [1]:
using DataFrames, Optim, Revise, Serialization, DebuggingUtilities

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__);
using BLPmodule; const m = BLPmodule;

In [3]:
# df = deserialize("jls/df.jls");
pars = deserialize("jls/pars.jls");
ec = deserialize("jls/ec.jls");

---

In [4]:
# linear characteristics (staffing variables):
X1 = vcat(transpose.([ec.firms[i].X for i in eachindex(ec.firms)])...)
Z = X1;

In [34]:
# gmm
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
m.gmm_lm([10.,10.], ec, pars, X1);


(ec.tracts[1]).shares = [1.0; 1.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:14)


(ec.tracts[1]).shares = [0.022796195884203563; 0.029423280902398825]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:18)


In [36]:
# analytic gradient
m.gmm_grad([1.,1.], ec, pars, X1)


(ec.tracts[1]).shares = [0.022796195884203563; 0.029423280902398825]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:89)
(ec.tracts[1]).shares = [0.022796195884203563; 0.029423280902398825]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:32)
0.022796195884203563
grad = 

[576.4668194822012, 4.727962279015691]


2-element Vector{Float64}:
 576.4668194822012
   4.727962279015691

In [26]:
# finite diff
m.grad_diff([1.,1.], ec, pars, X1)

θ2 = [1.0, 1.0]


θ2 = [1.00001, 1.0]


θ2 = [1.0, 1.00001]


2-element Vector{Float64}:
 1.0250903414998902
 0.005594466756519977

---

In [13]:
function g!(θ2::Array, storage::Array)
    storage .= m.gmm_grad(θ2, ec, pars, X1)
    return nothing
end


g! (generic function with 1 method)

In [15]:
# optim with analytic gradient
initial_θ2 = [1.,2.]
pars = deserialize("jls/pars.jls");
ec = deserialize("jls/ec.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), g!, initial_θ2, LBFGS())


(ec.tracts[1]).shares = [1.0; 1.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:32)
1.0
grad = 

[-1.0591817693673309, 0.009508595725479364]


 * Status: failure

 * Candidate solution
    Final objective value:     4.050715e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1


In [7]:
# optim with autodiff
initial_θ2 = [1.,1.]
pars = deserialize("jls/pars.jls");
ec = deserialize("jls/ec.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), initial_θ2, LBFGS(); autodiff = :forward)


(ec.tracts[1]).shares = [1.0; 1.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:14)


MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#13#14", Float64}, Float64, 2})
Closest candidates are:
  (::Type{T})(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  (::Type{T})(::T) where T<:Number at boot.jl:760
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number} at char.jl:50
  ...

In [12]:
# optim with finite diff
initial_θ2 = [1.,1.]
pars = deserialize("jls/pars.jls");
ec = deserialize("jls/ec.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), initial_θ2, LBFGS())


AssertionError: AssertionError: B > A

---

---